<a href="https://colab.research.google.com/github/isakdiaz/deeplabv3-tree-segmentation/blob/master/treeseg_coreml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CoreML Conversion from Tensorflow Model

In [9]:
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import pprint
import json
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
import json

import tensorflow_datasets as tfds

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.layers.experimental.preprocessing import Resizing
from tensorflow.keras.models import Model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing import image

from tensorflow.keras.callbacks import ModelCheckpoint


In [4]:
print(tf.__version__)
print(np.__version__)

2.6.0
1.19.5


## Load Cloud Drive

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Install CoreML Tools
### Requires Python 3.5 +

In [ ]:
!pip install coremltools

In [7]:
import coremltools as ct

In [8]:
!ls /content/drive/MyDrive/

 Biome_Videos	   'Interview Docs'	 saved_models
 Citizenships	    OakBark01_1024.png	 TreeGenerator.zip
'Colab Notebooks'   OakBark01.afphoto	 treeseg_coreml.ipynb
 CoreML		    plantnet


# Load Model File


In [15]:
# Check if a model exists
!ls -all -h /content/drive/MyDrive/saved_models/treeseg

total 206M
-rw------- 1 root root 206M Aug 29 16:09 treeseg2021-08-29.h5


## Load custom loss functions

In [21]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.utils import CustomObjectScope
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score
from tqdm import tqdm

def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

# Load Model

In [22]:
model_file = "/content/drive/MyDrive/saved_models/treeseg/treeseg2021-08-29.h5"
with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef, 'dice_loss': dice_loss}):
  model = tf.keras.models.load_model(model_file)

In [ ]:
model.summary()

# Convert Keras model to CoreML Model


In [30]:
# image_input = ct.ImageType(color_layout="RGB", scale=1/127.0, bias=[-1,-1,-1])
image_input = ct.ImageType(color_layout="RGB", scale=1/255.0, bias=[0,0,0])
# classifier_config = ct.ClassifierConfig(labels)

# Set input as ImageType so CoreML can automatically resize it using Vision framework
coreml_model = ct.convert(model, inputs=[image_input])

Translating MIL ==> MLModel Ops: 100%|██████████| 969/969 [00:02<00:00, 337.09 ops/s]


In [31]:
# # Define Spec Function
from coremltools.models.neural_network.builder import _get_nn_spec as get_nn

## Get Spec and check preprocessing
spec = coreml_model.get_spec()
nn = get_nn(spec)
print(nn.preprocessing)

[featureName: "input_3"
scaler {
  channelScale: 0.003921568859368563
}
]


## Save CoreML Model to Google Drive

In [35]:
# !ls -all -hs "$gdrive_dir"
model_name = model_file.split("/")[-1]
folder_name = gdrive_dir + "/CoreML"
!mkdir -p {folder_name}
coreml_file_path = "{0}/{1}.mlmodel".format(folder_name, model_name)
coreml_model.save(coreml_file_path)
print("Core ML model {} saved in {}".format(model_name, folder_name))

Core ML model treeseg2021-08-29.h5 saved in /content/drive/MyDrive/CoreML


In [ ]:
coreMLModel.get_spec

<bound method MLModel.get_spec of input {
  name: "input_2"
  type {
    imageType {
      width: 224
      height: 224
      colorSpace: RGB
      imageSizeRange {
        widthRange {
          lowerBound: 224
          upperBound: 224
        }
        heightRange {
          lowerBound: 224
          upperBound: 224
        }
      }
    }
  }
}
output {
  name: "Identity"
  type {
    multiArrayType {
      dataType: FLOAT32
    }
  }
}
metadata {
  userDefined {
    key: "com.github.apple.coremltools.source"
    value: "tensorflow==2.5.0"
  }
  userDefined {
    key: "com.github.apple.coremltools.version"
    value: "4.1"
  }
}
>